In [ ]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

os.environ['TOKEN'] = '' #Bearer token

def auth():
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keyword, start_date, end_date, max_results=500):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Endpoint
    #Query Parameters 
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id',
                    'tweet.fields': 'id,text,created_at,entities,source',
                    'user.fields': 'id,username,location',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def append_to_csv(json_response, fileName):
    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        for element in json_response['includes']['users']:
            
        # 1. Author ID
            author_id = tweet['author_id']
            user_id = element['id']
            if author_id == user_id:
                # 2. Time created
                created_at = dateutil.parser.parse(tweet['created_at'])
                username = element['username']
                # 8. Tweet text
                text = tweet['text'].replace("\n","").replace("\r","")
                source = tweet['source']
                if ('location' in element):   
                    location = element['location']
                else:
                    location = " "
                # Assemble all data in a list
                res = [author_id, created_at, username, text, source, location]
                # Append the result to the CSV file
                csvWriter.writerow(res)
                counter += 1
    csvFile.close()
    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 
with open('afghanistan_tweets_data.json', 'w') as f:
    json.dump(json_response, f)
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "(afghanistancrisis OR afghanistandisaster OR talibantakeover) lang:en -is:retweet -is:nullcast -has:links -has:mentions -has:media -has:images -is:reply "
start_list =    ['2021-08-01T00:00:00.000Z',
                 '2021-09-01T00:00:00.000Z',
                 '2021-10-01T00:00:00.000Z',
                 '2021-11-01T00:00:00.000Z',
                 '2021-12-01T00:00:00.000Z']

end_list =      ['2021-08-31T00:00:00.000Z',
                 '2021-09-30T00:00:00.000Z',
                 '2021-10-31T00:00:00.000Z',
                 '2021-11-30T00:00:00.000Z',
                 '2021-12-31T00:00:00.000Z']
max_results = 500
#Total number of tweets we collected from the loop
total_tweets = 0
# Create file
csvFile = open("afghanistan_tweets_data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
#Create headers
csvWriter.writerow(['author id', 'created_at', 'username', 'tweet', 'source', 'location'])
csvFile.close()
for i in range(0,len(start_list)):
    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 50000 # Max tweets per time period
    flag = True
    next_token = None
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i])
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "afghanistan_tweets_data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "afghanistan_tweets_data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            #Turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
    print("Total number of results: ", total_tweets) 